In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import pytz
from datetime import datetime, timedelta, date
import pandas as pd
from sklearn.preprocessing import LabelEncoder


# Define Argentina's timezone (GMT-3)
argentina_tz = pytz.timezone('America/Argentina/Buenos_Aires')

# Get the current date and time in Argentina
current_time_in_argentina = datetime.now(argentina_tz).replace(minute=0, second=0, microsecond=0).strftime('%Y-%m-%d %H:%M:%S')
# Round down (floor) to the nearest hour by setting minutes, seconds, and microseconds to 0
current_date = pd.to_datetime(current_time_in_argentina)
current_date

Timestamp('2024-10-02 19:00:00')

In [18]:
from src.inferencesm import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

# Create a label encoder object
label_encoder = LabelEncoder()

# Apply label encoding to 'line' and 'station'
features['line'] = label_encoder.fit_transform(features['line'])
features['station'] = label_encoder.fit_transform(features['station'])

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/603286
Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2024-09-19 17:00:00+00:00 to 2024-10-02 18:00:00+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.39s) 


ValueError: Time-series data is incomplete. Expected 14784 rows, but got 13772. Please ensure the feature pipeline is running properly.

In [ ]:
features

In [ ]:
from src.model_registry import get_latest_model_from_registry
from src.inferencesm import get_model_predictions

model = get_latest_model_from_registry(model_name='subwayBA_passenger_flow_updt', status= 'Production')
predictions = get_model_predictions(model, features)

In [ ]:
predictions['hour_of_entry'] = current_date
predictions

In [ ]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generated for the next 3 hours by our production model",
    primary_key = ['station', 'line', 'hour_of_entry'],
    event_time='hour_of_entry',
)

In [ ]:
predictions['station'] = predictions['station'].astype('int32')
predictions['line'] = predictions['line'].astype('int32')
print(predictions.dtypes)
feature_group.insert(predictions, write_options={"wait_for_job": False})